# WSI - ćwiczenie 6 - Uczenie się ze wzmocnieniem.
*Tomasz Frankowski*

Celem zadania była implementacja algorytmu Q-learning, oraz stworzenie agenta rozwiązującego problem Taxi.

In [2]:
import gym
import numpy as np
import random
from IPython.display import clear_output
from time import sleep

#### Importuje środowisko taxi
Celem zadanie będzie odebranie pasażera z jednego z punktów R, G, B, Y i odwiezienie go na wskazany punkt. Litera w kolorze niebieskim oznacza punkt startowy podróży pasażera, a w kolorze fioletowym oznacza punkt docelowy. Pozycja taksówki oznaczona jest przez żółty kwadrat lub zielony w przypadku kiedy przewozimy pasażera

In [3]:
env = gym.make("Taxi-v3").env
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Wszystkie możwliwe akcje oraz stany naszego środowiska

In [10]:
print(f"Wszyskie stany: {env.observation_space}")
print(f"Wszyskie akcje: {env.action_space}")

Wszyskie stany: Discrete(500)
Wszyskie akcje: Discrete(6)


Sposób poruszania taksówki

Dostepne akcje <br>
0 - ruch o jedna pozycje w dol <br>
1 - ruch o jedna pozycja w gore <br>
2 - ruch o jedna pozycje w prawo<br>
3 - ruch o jedna pozycje w lewo<br>
4 - odbior pasazera<br>
5 - zrzut pasazera


In [113]:
env.render()
env.step(2)
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)


### Algorytm Q-learning

In [4]:
def Q_learning(alpha, gamma, epsilon, iterations, q_table, environment):

    for i in range(1, iterations):
        # reset environment
        state = environment.reset()

        reward = 0
        done = False
        
        # Iterate over enviornemnt until goal is reached
        while not done:

            # Choose action, from learned or choose random
            if random.uniform(0, 1) < epsilon:
                action = environment.action_space.sample()
            else:
                action = np.argmax(q_table[state])

            # Do action
            next_state, reward, done, info = environment.step(action) 
            
            # Actualize Q-table
            prev_state = q_table[state, action]
            future_max_state = np.max(q_table[next_state])
            q_table[state, action]  = (1 - alpha) * prev_state + alpha * (reward + gamma * future_max_state)

            state = next_state
            
    return q_table

### Trenowanie agenta

In [12]:
# Example hiperparameters
alpha = 0.8
gamma = 0.6
epsilon = 0.7

q_tabela = Q_learning(alpha, gamma, epsilon, 100000, np.zeros([env.observation_space.n, env.action_space.n]), env)

Sprawdzam poprawność tabeli dla przykładowego stanu

In [127]:
env.reset()
env.render()
print(q_tabela[env.s])
next_action = np.argmax(q_tabela[env.s])
env.step(next_action)
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

[ -3.01034653  -2.67422442  -2.87192019  -3.01034997 -11.87013637
 -11.87144288]
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)


Jak widzimy wykonany ruch wydaje się być sensowny ponieważ zbliżamy się po pasażera

### Ewaluacja otrzymanych wyników

In [6]:
# Visualize taxi movement
def disp_frame(frame):
    clear_output(wait=True)
    print(frame)
    sleep(0.5)


def evaluate(episodes, q_table, environment, t_max, visualize=False):
    total_epochs = 0
    total_penalties = 0
    # Evaluation for each episode
    for _ in range(episodes):
        state = environment.reset()
        epochs = 0
        penalties = 0
        done = False
        t = 0
        while not done and t<t_max:
            # Choose best action
            action = np.argmax(q_table[state])
            # Make action
            state, reward, done, info = environment.step(action)

            if visualize:
                disp_frame(environment.render(mode='ansi'))

            # Check if taxi dropped passenger at wrong location
            if reward == -10:
                penalties += 1
            
            epochs += 1
            t += 1

        total_penalties += penalties
        total_epochs += epochs

    return episodes, total_epochs, total_penalties

Ewaluacja uzyskanego agenta

In [13]:
episodes, total_epochs, total_penalties = evaluate(1000, q_tabela, env, 100, False)

print(f"Episodes: {episodes}")
print(f"Avarage epochs per episode: {total_epochs/episodes}")
print(f"Avarage penalties per episode: {total_penalties/episodes}")

Episodes: 1000
Avarage epochs per episode: 13.086
Avarage penalties per episode: 0.0


Dla 10000 testów środowisko za każdym razem doszło do końcowego celu oraz taksówkarz ani razu się nie pomylił

### Test różnych hiperparametrów do wyznaczenia q-table

In [188]:
def grid_search(environment, step, q_learn_func, evaluate_func, learn_iterations, test_episodes, test_iterations):
    best_hiperparams = []
    alpha =0.0 +step
    gamma = 0.0+step
    epsilon = 0.0+step

    best_hiperparams = [alpha, gamma, epsilon];
    q_tab = Q_learning(alpha, gamma, epsilon, learn_iterations, np.zeros([environment.observation_space.n, environment.action_space.n]), environment)
    episodes, total_epochs, total_penalties  = evaluate(test_episodes, q_tab, environment, test_iterations, False)
    best_score = total_epochs; # the less the better
    best_penalties = total_penalties; # there should be no penalties


    while alpha < 1:
        gamma = 0 + step
        while gamma < 1:
            epsilon = 0 + step
            while epsilon < 1:

                q_tab = Q_learning(alpha, gamma, epsilon, learn_iterations, np.zeros([environment.observation_space.n, environment.action_space.n]), environment)
                episodes, total_epochs, total_penalties  = evaluate(test_episodes, q_tab, environment, test_iterations, False)


                if(total_epochs < best_score and total_penalties <= best_penalties) and total_epochs != 0:
                    best_score = total_epochs; # the less the better
                    best_penalties = total_penalties; # there should be no penalties
                    best_hiperparams = [alpha, gamma, epsilon]

                print(f"Testing: alpha { alpha:.2f}, gamma {gamma:.2f}, epsilon {epsilon:.2f}")
                print(f"Total epochs: {total_epochs}")
                print(f"Total penalties: {total_penalties}")
                epsilon += step

            gamma += step
            
        alpha += step

    return best_hiperparams
            

In [189]:
best_params = grid_search(env, 0.1, Q_learning, evaluate, 1000, 10, 100)

Testing: alpha 0.10, gamma 0.10, epsilon 0.10
Total epochs: 1000
Total penalties: 0
Testing: alpha 0.10, gamma 0.10, epsilon 0.20
Total epochs: 1000
Total penalties: 0
Testing: alpha 0.10, gamma 0.10, epsilon 0.30
Total epochs: 1000
Total penalties: 0
Testing: alpha 0.10, gamma 0.10, epsilon 0.40
Total epochs: 1000
Total penalties: 0
Testing: alpha 0.10, gamma 0.10, epsilon 0.50
Total epochs: 1000
Total penalties: 0
Testing: alpha 0.10, gamma 0.10, epsilon 0.60
Total epochs: 1000
Total penalties: 0
Testing: alpha 0.10, gamma 0.10, epsilon 0.70
Total epochs: 817
Total penalties: 0
Testing: alpha 0.10, gamma 0.10, epsilon 0.80
Total epochs: 731
Total penalties: 0
Testing: alpha 0.10, gamma 0.10, epsilon 0.90
Total epochs: 729
Total penalties: 0
Testing: alpha 0.10, gamma 0.10, epsilon 1.00
Total epochs: 142
Total penalties: 0
Testing: alpha 0.10, gamma 0.20, epsilon 0.10
Total epochs: 1000
Total penalties: 0
Testing: alpha 0.10, gamma 0.20, epsilon 0.20
Total epochs: 812
Total penalties:

Uzyskane najlepsze parametry po przeszukaniu wszystkich kombinacji hiperparametrów z krokiem równym 0.1.

In [14]:
# Best parameters
alpha = 0.7
gamma = 0.2
epsilon = 0.8

Sprawdzam czy otrzymane hiperparamatery są faktycznie lepsze od pierwotnych

In [15]:
q_tabela = Q_learning(alpha, gamma, epsilon, 100000, np.zeros([env.observation_space.n, env.action_space.n]), env)
episodes, total_epochs, total_penalties = evaluate(1000, q_tabela, env, 100, False)

print(f"Episodes: {episodes}")
print(f"Avarage epochs per episode: {total_epochs/episodes}")
print(f"Avarage penalties per episode: {total_penalties/episodes}")

Episodes: 1000
Avarage epochs per episode: 13.046
Avarage penalties per episode: 0.0


Otrzymany wynik jest nieznacznie lepszy niż dla poprzednich hiperparametrów. Może mieć na to wpływ to że w przypadku przeszukania wszystkich hiperparametrów uczyliśmy agenta przez zaledwie 1000 różnych kombinacji, a w przypadku już ostatecznego testu użyliśmy aż 100000 różnych scenariuszy. Po paru testach zauważyłem że w przypadku aż tylu iteracji większość kombinacji hiperparametrów będzie dobrze działała a średni czas odcinka wynosić będzie 13. Sprawdze w takim razie czy w przypadku użycia 1000 scenariuszy coś się zmieni.

Dla pierwotnych hiperparametrów

In [22]:
# Example hiperparameters
alpha = 0.8
gamma = 0.6
epsilon = 0.7
scenarios = 1000
q_tabela = Q_learning(alpha, gamma, epsilon, scenarios, np.zeros([env.observation_space.n, env.action_space.n]), env)
episodes, total_epochs, total_penalties = evaluate(1000, q_tabela, env, 100, False)

print(f"Episodes: {episodes}")
print(f"Avarage epochs per episode: {total_epochs/episodes}")
print(f"Avarage penalties per episode: {total_penalties/episodes}")

Episodes: 1000
Avarage epochs per episode: 15.607
Avarage penalties per episode: 0.0


Dla uzyskanych hiperparametrów

In [23]:
# Best parameters
alpha = 0.7
gamma = 0.2
epsilon = 0.8
scenarios = 1000
q_tabela = Q_learning(alpha, gamma, epsilon, scenarios, np.zeros([env.observation_space.n, env.action_space.n]), env)
episodes, total_epochs, total_penalties = evaluate(1000, q_tabela, env, 100, False)

print(f"Episodes: {episodes}")
print(f"Avarage epochs per episode: {total_epochs/episodes}")
print(f"Avarage penalties per episode: {total_penalties/episodes}")

Episodes: 1000
Avarage epochs per episode: 13.87
Avarage penalties per episode: 0.0


Otrzymany wynik dla najlepszych hiperparametrów jest lepszy niż dla pierwotnych. Co najważniejsze czas uczenia agenta zmniejszył się do zaledwie 5s w porównaniu do poprzedniego przy tescie dla 100000 scenariuszy gdzie wynosił on około 1m. Mimo zmniejszenia ilości scenariuszy uczących wynik nie uległ znacznemu pogorszeniu.